# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 🚀 Project: YouTube Automation OS

### A Full-Stack Data Science Application
This project is an advanced analytics dashboard designed for YouTube creators. It uses the YouTube Data API, Natural Language Processing (NLP), and Machine Learning to analyze video performance, estimate earnings, and spy on competitor tags.

---

# 🔴 [CLICK HERE TO LAUNCH LIVE APP](https://youtube-automation-os-3rc6jevskgnnpijdq3de9a.streamlit.app/)

---

### 🛠️ Tech Stack
* **Python** (Core Logic)
* **Streamlit** (User Interface)
* **YouTube Data API v3** (Data Mining)
* **TextBlob** (Sentiment Analysis)
* **Seaborn/Matplotlib** (Visualization)

### 🌟 Key Features
1.  **Revenue Estimator:** Calculates potential AdSense earnings based on RPM.
2.  **Virality Score:** A custom algorithm (0-100) to rate video performance.
3.  **Tag Spy:** Extracts hidden SEO tags from top-performing videos.
4.  **Sentiment Analysis:** Detects if titles are Positive, Negative, or Neutral.

In [1]:
# --- CELL 1: INSTALL LIBRARIES ---

# I removed the broken "youtube-api-python-client" library. 
# We only need the "google-api-python-client".
!pip install streamlit google-api-python-client textblob wordcloud seaborn matplotlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.4 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from googleapiclient.discovery import build
from textblob import TextBlob
from wordcloud import WordCloud
from collections import Counter
import os

# --- PAGE CONFIG ---
st.set_page_config(page_title="YouTube Automation OS", page_icon="⚡", layout="wide")

# --- CUSTOM CSS ---
st.markdown("""
    <style>
    .big-font { font-size:24px !important; font-weight: bold; }
    .metric-card { background-color: #f0f2f6; padding: 15px; border-radius: 10px; border-left: 5px solid #ff0000; }
    </style>
""", unsafe_allow_html=True)

# --- SIDEBAR ---
with st.sidebar:
    st.title("⚡ Automation OS")
    api_key = st.text_input("🔑 API Key", type="password")
    st.divider()
    country_code = st.selectbox("Target Region", ["US", "IN", "GB", "CA", "AU"], index=0)
    rpm = st.slider("Est. RPM ($)", 0.5, 10.0, 2.5)

# --- FUNCTIONS ---
def get_data(api_key, query, max_results=50):
    youtube = build('youtube', 'v3', developerKey=api_key)
    search_req = youtube.search().list(
        part="snippet", q=query, type="video", 
        regionCode=country_code, maxResults=max_results, order="viewCount"
    )
    search_res = search_req.execute()
    video_ids = [item['id']['videoId'] for item in search_res.get('items', [])]
    
    stats_req = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids))
    stats_res = stats_req.execute()
    
    data = []
    all_tags = []
    
    for item in stats_res.get('items', []):
        stats = item['statistics']
        snippet = item['snippet']
        views = int(stats.get('viewCount', 0))
        likes = int(stats.get('likeCount', 0))
        comments = int(stats.get('commentCount', 0))
        title = snippet['title']
        tags = snippet.get('tags', [])
        all_tags.extend(tags)
        
        engagement = ((likes + comments) / views * 100) if views > 0 else 0
        revenue = (views / 1000) * rpm
        blob = TextBlob(title)
        sentiment = "Positive" if blob.sentiment.polarity > 0 else "Neutral"
        raw_score = (views * 0.7) + (likes * 50) + (comments * 100)
        
        data.append({
            'Thumbnail': snippet['thumbnails']['high']['url'],
            'Title': title,
            'Views': views,
            'Likes': likes,
            'Engagement': round(engagement, 2),
            'Est. Earnings ($)': round(revenue, 2),
            'Sentiment': sentiment,
            'Title Len': len(title),
            'Published': snippet['publishedAt'],
            'Raw_Score': raw_score
        })
    
    df = pd.DataFrame(data)
    if not df.empty:
        df['Virality Score'] = (df['Raw_Score'] / df['Raw_Score'].max()) * 100
        df['Virality Score'] = df['Virality Score'].round(1)
    
    return df, all_tags

# --- MAIN UI ---
st.title("⚡ YouTube Automation OS")
col1, col2 = st.columns([3, 1])
with col1:
    query = st.text_input("Enter Niche / Topic")
with col2:
    st.write("") 
    st.write("") 
    search_btn = st.button("🚀 Launch", use_container_width=True, type="primary")

if search_btn and api_key:
    with st.spinner('Analyzing...'):
        try:
            df, all_tags = get_data(api_key, query)
            m1, m2, m3, m4 = st.columns(4)
            m1.metric("Total Views", f"{df['Views'].sum():,}")
            m2.metric("Total Earnings", f"${df['Est. Earnings ($)'].sum():,.0f}")
            m3.metric("Avg Virality", f"{df['Virality Score'].mean():.0f}/100")
            m4.metric("Title Len", f"{int(df['Title Len'].mean())}")
            
            st.divider()
            t1, t2, t3 = st.tabs(["Tag Spy", "Top Videos", "Analytics"])
            
            with t1:
                tag_counts = Counter(all_tags).most_common(15)
                st.text_area("📋 Copy Tags", ", ".join([t[0] for t in tag_counts]))
                wc = WordCloud(width=600, height=300, background_color='white').generate_from_frequencies(dict(tag_counts))
                fig, ax = plt.subplots()
                plt.imshow(wc, interpolation='bilinear')
                plt.axis("off")
                st.pyplot(fig)
            
            with t2:
                st.dataframe(
                    df[['Thumbnail', 'Title', 'Views', 'Virality Score']].sort_values('Virality Score', ascending=False),
                    column_config={"Thumbnail": st.column_config.ImageColumn("Preview"), "Virality Score": st.column_config.ProgressColumn("Score", min_value=0, max_value=100)},
                    use_container_width=True
                )
            
            with t3:
                fig, ax = plt.subplots()
                sns.scatterplot(data=df, x='Views', y='Est. Earnings ($)', hue='Sentiment', size='Virality Score', ax=ax)
                st.pyplot(fig)

        except Exception as e:
            st.error(f"Error: {e}")

Writing app.py


In [3]:
# --- CELL 3: RUN THE APP & GET PUBLIC LINK ---
import subprocess

# 1. Run Streamlit in the background
print("Starting Streamlit...")
# We run it on port 8501
process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# 2. Get your Public IP (You need this for the password)
print("Your Password is:")
!curl ipv4.icanhazip.com

# 3. Create the public tunnel
print("\nCreating Public Link... (Click the link below and enter the password above)")
!npm install -g localtunnel > /dev/null
!npx localtunnel --port 8501

Starting Streamlit...
Your Password is:
34.73.77.47

Creating Public Link... (Click the link below and enter the password above)



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.73.77.47:8501

npm notice
npm notice New major version of npm available! 10.8.2 -> 11.6.2
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.6.2
npm notice To update run: npm install -g npm@11.6.2
npm notice
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://shiny-rules-notice.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:19161 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.proces